Generating Simulation Data
==========================

This notebook contains all the scripts we used to generate simulated data which fed into the figures in the manuscript.

Keep in mind that all the data that results from these sweeps has been generated already - if you would like
to look at and work with the exact simulation results that we used to generate the figures in the paper, you
do not need to rerun these cells (although you certainly can!). Just be careful not to accidentally overwrite
the pregenerated data.

In [1]:
from RPI_tools import pytorch_tools, tf_tools
import time
import numpy as np
import torch as t
from tqdm import tqdm
import os

# Edit this line to point to the location of the data folder on your system.
data_folder_prefix = 'data/'

Sweep over R
------------

The following cells generate simulated data, and do the relevant iterative reconstructions, for a sweep
over the resolution ratio R at fixed photon fluence. The first cell sets the parameters for the sweep, the
second cell runs the simulations, and the third cell runs the relevant iterative reconstructions.

In [2]:
prefix = data_folder_prefix + 'Simulated/R_Sweep/'

#check if the folder exist
isExist = os.path.exists(prefix)
if not isExist:
    # Create a new directory if it does not exist 
    os.makedirs(prefix)
    print("The new directory is created!")

Rs = [0.25, 0.5, 1, 2] # The set of resolution ratios to study
training = 100 #4000 # The number of training images to generate
testing = 100 # The number of testing images to generate
photon_level = 1e4 # The photon level to study, per pixel in the RPI reconstruction

In [3]:
for R in Rs:
        
    # The maximum frequency contained in the probe, in pixels, for the given
    # resolution ratio and and object size of 128
    probe_maxk = 128//R

    # We then calculate the size of the array on which we have to simulate
    # the probe to capture all the frequency components. 
    field_size = int(np.ceil(probe_maxk) + 1) * 2

    # This generates an ideal BLR probe with the given parameters
    probe = tf_tools.generate_blr_probe([field_size,field_size], probe_maxk)
    
    # ABE - what is the meaning of that random-seeming number used to divide through the value of "peak"?
    # We generate a set of training data - diffraction patterns and ground truth images.
    expanded_probe, (tr_patterns, tr_images), (test_patterns, test_images) = tf_tools.generate_imagenet_phase_data(
        probe, n_train=training, n_test = testing, n_photons_per_pix = photon_level, data_folder_prefix=data_folder_prefix)
    
    print('Data Generated')

    num_tr = tr_patterns.shape[0]
    num_test = test_patterns.shape[0]
    num_rows = tr_patterns.shape[1]
    num_cols = tr_patterns.shape[2]
    
    print('Training Pattern Dimensions:', num_tr, 'by', num_rows, 'by', num_cols)
    print('Testing Pattern Dimensions:', num_test, 'by', num_rows, 'by', num_cols)

    # This will overwrite the previous files at each photon level, but all photon levels
    # use the same images & probe so it is not an issue other than being inefficient.
    np.save(prefix + 'test_images-R-%0.2f.npy' % R, test_images)
    np.save(prefix + 'tr_images-R-%0.2f.npy' % R, tr_images)
    np.save(prefix + 'probe-R-%0.2f.npy' % R, expanded_probe)
    
    # We do save out the patterns in separate files as they are afffected by the photon level.
    np.save(prefix + 'test_patterns-R-%0.2f-phperpix-%d.npy' % (R, photon_level), test_patterns)
    np.save(prefix + 'tr_patterns-R-%0.2f-phperpix-%d.npy' % (R, photon_level), tr_patterns)
    
    print('Simulated Data for R=%0.2f Saved' % R)

Data Generated
Training Pattern Dimensions: 100 by 1282 by 1282
Testing Pattern Dimensions: 100 by 1282 by 1282
Simulated Data for R=0.25 Saved
Data Generated
Training Pattern Dimensions: 100 by 770 by 770
Testing Pattern Dimensions: 100 by 770 by 770
Simulated Data for R=0.50 Saved
Data Generated
Training Pattern Dimensions: 100 by 514 by 514
Testing Pattern Dimensions: 100 by 514 by 514
Simulated Data for R=1.00 Saved
Data Generated
Training Pattern Dimensions: 100 by 386 by 386
Testing Pattern Dimensions: 100 by 386 by 386
Simulated Data for R=2.00 Saved


In [4]:
# ABE - why this particular pattern of checkpoints?
reconstruction_checkpoints = [1, 10] #[1, 5, 10, 50, 100, 200, 300, 500, 1000]

lr = 0.5 # set the learning rate for iterative reconstruction

for R in Rs:

    print('Working on Resolution Ratio', R)
    expanded_probe = t.from_numpy(np.load(prefix + 'probe-R-%0.2f.npy' % R))

    print('Generating Approximants from Training Data')
    tr_patterns = t.from_numpy(np.load(prefix + 'tr_patterns-R-%0.2f-phperpix-%d.npy' % (R, photon_level)))
    
    tr_approximants = []
    for pattern in tqdm(tr_patterns):
        # This performs a single iteration of the reconstruction algorithm with steepest gradient descent
        # for training the deep k-learning framework
        approximant, _ = pytorch_tools.reconstruct(pattern, expanded_probe, resolution=256, lr=1, iterations=1, 
                                                   loss_func=pytorch_tools.amplitude_mse, GPU=True)
        tr_approximants.append(np.angle(approximant.numpy()))

    tr_approximants = np.array(tr_approximants)
    np.save(prefix + 'tr-reconstruction-R-%0.2f-phperpix-%d-iters-%d-lr-%0.2f.npy' 
            % (R, photon_level, 1, 1), tr_approximants)
    
    print('Generating Approximants from Testing Data')
    test_patterns = t.from_numpy(np.load(prefix + 'test_patterns-R-%0.2f-phperpix-%d.npy' % (R, photon_level)))
    
    test_approximants = []
    for pattern in tqdm(test_patterns):
        # This performs a single iteration of the reconstruction algorithm with steepest gradient descent
        # for testing the deep k-learning framework
        approximant, _ = pytorch_tools.reconstruct(pattern, expanded_probe, resolution=256, lr=1, iterations=1, 
                                                   loss_func=pytorch_tools.amplitude_mse, GPU=True)
        test_approximants.append(np.angle(approximant.numpy()))
        
    test_approximants = np.array(test_approximants)
    np.save(prefix + 'test-reconstruction-R-%0.2f-phperpix-%d-iters-%d-lr-%0.2f.npy' 
            % (R, photon_level, 1, 1), test_approximants)
    
    print('Performing Reconstructions on Testing Data')
    # Store runtime information for each iteration step
    runtimes = np.zeros([2,len(reconstruction_checkpoints)]) 
    for i, iters in enumerate(reconstruction_checkpoints):
        
        print('Doing Reconstructions with %d iterations' % iters)
        
        test_results = []
        start = time.time()
        for pattern in tqdm(test_patterns):
            # This performs <iters> iterations of the reconstruction algorithms
            result, _ = pytorch_tools.reconstruct(pattern, expanded_probe, resolution=256, 
                                                  lr=lr, iterations=iters, loss_func=pytorch_tools.amplitude_mse, GPU=True)
            test_results.append(np.angle(result.numpy()))
            
        runtimes[:,i] = [iters, time.time() - start] # The format is [# iterations, runtime] for each checkpoint

        test_results = np.array(test_results)
        np.save(prefix + 'test-reconstruction-R-%0.2f-phperpix-%d-iters-%d-lr-%0.2f.npy' % (R, photon_level, iters, lr), test_results)
    
    np.save(prefix + 'runtimes-R-%0.2f-phperpix-%d-lr-%0.2f.npy' % (R, photon_level, lr), runtimes)

Working on Resolution Ratio 0.25
Generating Approximants from Training Data


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Generating Approximants from Testing Data


  3%|██▍                                                                               | 3/100 [00:00<00:03, 28.11it/s]

Performing Reconstructions on Testing Data
Doing Reconstructions with 1 iterations


...

 18%|██████████████▍                                                                 | 18/100 [00:00<00:00, 173.54it/s]

Performing Reconstructions on Testing Data
Doing Reconstructions with 1 iterations


  4%|███▎                                                                              | 4/100 [00:00<00:02, 38.94it/s]

Doing Reconstructions with 10 iterations


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 38.97it/s]


Sweep over Photon Shot Noise at Fixed R
---------------------------------------

The following cells generate simulated data, and do the relevant iterative reconstructions, for a sweep
over the photon fluence at a fixed resolution rato R. The first cell sets the parameters for the sweep, the
second cell runs the simulations, and the third cell runs the relevant iterative reconstructions.

In [5]:
prefix = data_folder_prefix + 'Simulated/Fixed_R_Noise_Sweep/'

#check if the folder exist
isExist = os.path.exists(prefix)
if not isExist:
    # Create a new directory if it does not exist 
    os.makedirs(prefix)
    print("The new directory is created!")

R = 0.5 # The resolution ratio to study
training = 100 #4000 # The number of training images to generate
testing = 100 # The number of testing images to generate
photon_levels = [1e-2, 1e-1, 1, 1e1, 1e2, 1e3] # The set of photon levels to study, per pixel in the RPI reconstruction

In [6]:
for photon_level in photon_levels:
        
    # The maximum frequency contained in the probe, in pixels, for the given
    # resolution ratio and and object size of 128
    probe_maxk = 128//R

    # We then calculate the size of the array on which we have to simulate
    # the probe to capture all the frequency components. 
    field_size = int(np.ceil(probe_maxk) + 1) * 2

    # This generates an ideal BLR probe with the given parameters
    probe = tf_tools.generate_blr_probe([field_size,field_size], probe_maxk)
    
    # ABE - what is the meaning of that random-seeming number used to divide through the value of "peak"?
    # We generate a set of training data - diffraction patterns and ground truth images.
    expanded_probe, (tr_patterns, tr_images), (test_patterns, test_images) = tf_tools.generate_imagenet_phase_data(
        probe, n_train=training, n_test = testing, n_photons_per_pix = photon_level, data_folder_prefix=data_folder_prefix)
    
    print('Data Generated')

    num_tr = tr_patterns.shape[0]
    num_test = test_patterns.shape[0]
    num_rows = tr_patterns.shape[1]
    num_cols = tr_patterns.shape[2]
    
    print('Training Pattern Dimensions:', num_tr, 'by', num_rows, 'by', num_cols)
    print('Testing Pattern Dimensions:', num_test, 'by', num_rows, 'by', num_cols)

    # This will overwrite the previous files at each photon level, but all photon levels
    # use the same images & probe so it is not an issue other than being inefficient.
    np.save(prefix + 'test_images-R-%0.2f.npy' % R, test_images)
    np.save(prefix + 'tr_images-R-%0.2f.npy' % R, tr_images)
    np.save(prefix + 'probe-R-%0.2f.npy' % R, expanded_probe)
    
    # We do save out the patterns in separate files as they are afffected by the photon level.
    np.save(prefix + 'test_patterns-R-%0.2f-phperpix-%0.2f.npy' % (R, photon_level), test_patterns)
    np.save(prefix + 'tr_patterns-R-%0.2f-phperpix-%0.2f.npy' % (R, photon_level), tr_patterns)
    
    print('Simulated Data for Photon Level %0.2f Saved' % photon_level)

Data Generated
Training Pattern Dimensions: 100 by 770 by 770
Testing Pattern Dimensions: 100 by 770 by 770
Simulated Data for Photon Level 0.01 Saved
Data Generated
Training Pattern Dimensions: 100 by 770 by 770
Testing Pattern Dimensions: 100 by 770 by 770
Simulated Data for Photon Level 0.10 Saved
Data Generated
Training Pattern Dimensions: 100 by 770 by 770
Testing Pattern Dimensions: 100 by 770 by 770
Simulated Data for Photon Level 1.00 Saved
Data Generated
Training Pattern Dimensions: 100 by 770 by 770
Testing Pattern Dimensions: 100 by 770 by 770
Simulated Data for Photon Level 10.00 Saved
Data Generated
Training Pattern Dimensions: 100 by 770 by 770
Testing Pattern Dimensions: 100 by 770 by 770
Simulated Data for Photon Level 100.00 Saved
Data Generated
Training Pattern Dimensions: 100 by 770 by 770
Testing Pattern Dimensions: 100 by 770 by 770
Simulated Data for Photon Level 1000.00 Saved


In [8]:
# ABE - why this particular pattern of checkpoints?
reconstruction_checkpoints = [1, 5]#, 10, 50, 100, 200, 300, 500, 1000]
lr = 0.5

for photon_level in photon_levels:

    print('Working on Photon Level', photon_level)
    expanded_probe = t.from_numpy(np.load(prefix + 'probe-R-%0.2f.npy' % R))

    print('Generating Approximants from Training Data')
    tr_patterns = t.from_numpy(np.load(prefix + 'tr_patterns-R-%0.2f-phperpix-%0.2f.npy' % (R, photon_level)))
    
    tr_approximants = []
    for pattern in tqdm(tr_patterns):
        # This performs a single iteration of the reconstruction algorithm
        approximant, _ = pytorch_tools.reconstruct(pattern, expanded_probe, 256, 
                                                   lr=1, iterations=1, loss_func=pytorch_tools.amplitude_mse, GPU=True)
        tr_approximants.append(np.angle(approximant.numpy()))

    tr_approximants = np.array(tr_approximants)
    np.save(prefix + 'tr-reconstruction-R-%0.2f-phperpix-%0.2f-iters-%d-lr-%0.2f.npy' % (R, photon_level, 1, 1), tr_approximants)
    
    print('Generating Approximants from Testing Data')
    test_patterns = t.from_numpy(np.load(prefix + 'test_patterns-R-%0.2f-phperpix-%0.2f.npy' % (R, photon_level)))
    test_approximants = []
    for pattern in tqdm(test_patterns):
        # This performs a single iteration of the reconstruction algorithm
        approximant, _ = pytorch_tools.reconstruct(pattern, expanded_probe, 256, 
                                                   lr=1, iterations=1, loss_func=pytorch_tools.amplitude_mse, GPU=True)
        test_approximants.append(np.angle(approximant.numpy()))

    test_approximants = np.array(test_approximants)
    np.save(prefix + 'test-reconstruction-R-%0.2f-phperpix-%0.2f-iters-%d-lr-%0.2f.npy' % (R, photon_level, 1, 1), test_approximants)
    
    print('Performing Reconstructions on Testing Data')
    runtimes = np.zeros([2,len(reconstruction_checkpoints)]) # This will store runtime information for each iteration step
    
    for i, iters in enumerate(reconstruction_checkpoints):
        
        print('Doing Reconstructions with %d iterations' % iters)
        
        test_results = []
        start = time.time()
        for pattern in tqdm(test_patterns):
            # This performs <iters> iterations of the reconstruction algorithms
            result, _ = pytorch_tools.reconstruct(pattern, expanded_probe, 256, lr=lr, 
                                                  iterations=iters, loss_func=pytorch_tools.amplitude_mse, GPU=True)
            test_results.append(np.angle(result.numpy()))
            
        runtimes[:,i] = [iters, time.time() - start] # The format is [# iterations, runtime] for each checkpoint

        test_results = np.array(test_results)
        np.save(prefix + 'test-reconstruction-R-%0.2f-phperpix-%0.2f-iters-%d-lr-%0.2f.npy' % (R, photon_level, iters, lr), test_results)
    
    np.save(prefix + 'runtimes-R-%0.2f-phperpix-%0.2f-lr-%0.2f.npy' % (R, photon_level, lr), runtimes)

 10%|████████                                                                         | 10/100 [00:00<00:00, 91.94it/s]

Working on Photon Level 0.01
Generating Approximants from Training Data


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Generating Approximants from Testing Data


 18%|██████████████▌                                                                  | 18/100 [00:00<00:00, 90.57it/s]

...

 11%|████████▊                                                                       | 11/100 [00:00<00:00, 104.05it/s]

Performing Reconstructions on Testing Data
Doing Reconstructions with 1 iterations


  5%|████                                                                              | 5/100 [00:00<00:02, 46.85it/s]

Doing Reconstructions with 5 iterations


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 45.47it/s]
